# Forward Planner

## Unify

This notebook uses the accompanying `unification.py` file for unification.

In [1]:
from unification import parse, unification, is_variable
from copy import deepcopy
from typing import Dict, List, Any, Tuple, Callable

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that.

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)",
    "(at Money Bank)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)",
    "(at Money Bank)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". 

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester applies, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...a list of actions, fully instantiated, for the agent to do in order: [a1, a2, a3]. If you pass an extra intermediate=True parameter, it should also return the resulting state of each action: [s0, a1, s1, a2, s2, a3, s3].

(Above assignment description and prompt by S. Butcher, 2022)
-----

<a id="parse_values"></a>
## parse_values
*The parse_values function converts all s-expressions in the starting state, goals, and actions into Python lists.* **Used by**: [forward_planner](#forward_planner)

* **start_state** List[str]: the starting state containing a list of s-expression
* **goals** List[str]: the goal state containing a list of s-expressions
* **actions** Dict[str, Any]: the actions dictionary containing actions, preconditions, and add and delete rules for each action.

**returns** Tuple[List[List[str]], List[List[str]], Dict[str, Any]].

In [2]:
def parse_values(start_state: List[str], goals: List[str], actions: Dict[str, Any]) -> Tuple[List[List[str]], List[List[str]], Dict[str, Any]]:
    new_actions = {}
    new_state = [parse(state) for state in start_state]
    new_goals = [parse(state) for state in goals]
    for action in actions.keys():
        new_actions[action] = {}
        new_actions[action]["action"] = parse(actions[action]["action"])
        new_actions[action]["add"] = [parse(add) for add in actions[action]["add"]]
        new_actions[action]["delete"] = [parse(delete) for delete in actions[action]["delete"]]
        new_actions[action]["conditions"] = [parse(condition) for condition in actions[action]["conditions"]]
    
    return new_state, new_goals, new_actions

In [3]:
#assertions/unit tests
test_start_state = [
    "(plane 1973)", 
    "(plane 2749)", 
    "(plane 97)"
]

test_goals = [
    "(at 1973 ORD)",
    "(at 1211 JFK)", 
    "(at 2749 SFO)"
]

test_actions = {
    "fly": {
        "action": "(fly ?plane ?from ?to)",
        "conditions": [
            "(plane ?plane)", 
            "(airport ?to)",
            "(airport ?from)",
            "(at ?plane ?from)" 
        ],
        "add": [
            "(at ?plane ?to)"
        ],
        "delete": [
            "(at ?plane ?from)"
        ]
    }
}

state, goals, actions = parse_values(test_start_state, test_goals, test_actions)

expected_state = [
    ["plane", "1973"], 
    ["plane", "2749"], 
    ["plane", "97"] 
]
assert state == expected_state

expected_goals = [
    ["at", "1973", "ORD"],
    ["at", "1211", "JFK"], 
    ["at", "2749", "SFO"]
]

assert goals == expected_goals

expected_actions = {
    "fly": {
        "action": ["fly", "?plane", "?from", "?to"],
        "conditions": [
            ["plane", "?plane"], 
            ["airport", "?to"],
            ["airport", "?from"],
            ["at", "?plane", "?from"] 
        ],
        "add": [
            ["at", "?plane", "?to"]
        ],
        "delete": [
            ["at", "?plane", "?from"]
        ]
    }
}

assert actions == expected_actions

<a id="get_action_successors"></a>
## get_action_successors
*The get_action_successors function is used by the inner breadth-first search that finds all permissible actions. This function finds all child nodes for a single precondition and the current state. The function returns a list of child nodes containing a substitution list and the remaining preconditions with the tested condition removed.* **Used by**: [get_permissible_actions](#get_permissible_actions)

* **action_node** Dict[str, Any]: an action node of the form {"substitutions": [], "conditions": []} where substitutions is a substitution list containing valid substitutions for this path and the conditions are the preconditions for the current action.
* **state** List[List[str]]: the current state

**returns** List[Dict[str, Any]]

In [4]:
def get_action_successors(action_node: Dict[str, Any], state: List[List[str]]) -> List[Dict[str, Any]]:
    children = []
    if len(action_node["conditions"]) == 0:
        return children
    current_condition = action_node["conditions"][0]
    for value in state:
        substitutions = unification(current_condition, value)
        if not substitutions:
            continue
        # merge unifications with existing substitutions and remove conflicts 
        updated_substitutions = None if any(key in action_node["substitutions"] and 
                                            substitutions[key] != action_node["substitutions"][key]
                                            for key in substitutions) else {**action_node["substitutions"], **substitutions}
        if updated_substitutions != None:
            child = {"substitutions": updated_substitutions, "conditions": action_node["conditions"][1:]}
            children.append(child)
    return children

In [5]:
#assertions/unit tests
test_start_state = [
    ["plane", "1973"], 
    ["plane", "2749"], 
    ["plane", "97"], 
    ["plane", "1211"], 
    ["airport", "SFO"], 
    ["airport", "JFK"], 
    ["airport", "ORD"], 
    ["at", "1973", "SFO"], 
    ["at", "2749", "JFK"], 
    ["at", "97", "ORD"],
    ["at", "1211", "SFO"], 
    ["fueled", "1973"], 
    ["unfueled", "2749"], 
    ["unfueled", "97"], 
    ["fueled", "1211"]
]

test_actions = {
     "fly": {
        "action": ["fly", "?plane", "?from", "?to"],
        "conditions": [
            ["plane", "?plane"], 
            ["airport", "?to"],
            ["airport", "?from"],
            ["at", "?plane", "?from"] 
        ],
        "add": [
            ["at", "?plane", "?to"]
        ],
        "delete": [
            ["at", "?plane", "?from"]
        ]
    },
    "fuel": {
        "action": ["fuel", "?plane"],
        "conditions": [
            ["plane", "?plane"],
            ["unfueled", "?plane"],
        ],
        "add": [
            ["fueled", "?plane"]
        ],
        "delete": [
            ["unfueled", "?plane"]
        ]
    }
}

current_node = {"substitutions": {}, "conditions": test_actions["fly"]["conditions"]}

# the expected response is a list of possible unifications for the first precondition in the 
# list and an updated contitions list with the first condition removed.
expected_resp_1 = [
    {'substitutions': {'?plane': '1973'},
       'conditions': [['airport', '?to'],
       ['airport', '?from'],
       ['at', '?plane', '?from']]},
    {'substitutions': {'?plane': '2749'},
        'conditions': [['airport', '?to'],
       ['airport', '?from'],
       ['at', '?plane', '?from']]},
    {'substitutions': {'?plane': '97'},
        'conditions': [['airport', '?to'],
       ['airport', '?from'],
       ['at', '?plane', '?from']]},
    {'substitutions': {'?plane': '1211'},
        'conditions': [['airport', '?to'],
       ['airport', '?from'],
       ['at', '?plane', '?from']]}
]

assert get_action_successors(current_node, test_start_state) == expected_resp_1

expected_resp_2 = [
    {'substitutions': {'?plane': '1973', '?to': 'SFO'}, 
     'conditions': [['airport', '?from'], ['at', '?plane', '?from']]}, 
    {'substitutions': {'?plane': '1973', '?to': 'JFK'}, 
     'conditions': [['airport', '?from'], ['at', '?plane', '?from']]}, 
    {'substitutions': {'?plane': '1973', '?to': 'ORD'}, 
     'conditions': [['airport', '?from'], ['at', '?plane', '?from']]}
]

assert get_action_successors(expected_resp_1[0], test_start_state) == expected_resp_2

expected_resp_3 = [
    {'substitutions': {'?plane': '1973', '?to': 'SFO', '?from': 'SFO'}, 
     'conditions': [['at', '?plane', '?from']]}, 
    {'substitutions': {'?plane': '1973', '?to': 'SFO', '?from': 'JFK'}, 
     'conditions': [['at', '?plane', '?from']]}, 
    {'substitutions': {'?plane': '1973', '?to': 'SFO', '?from': 'ORD'}, 
     'conditions': [['at', '?plane', '?from']]}
]

assert get_action_successors(expected_resp_2[0], test_start_state) == expected_resp_3

<a id="get_permissible_actions"></a>
## get_permissible_actions
*The get_permissible actions function applies a breadth-first search to return all permissible actions of the current state for a given action. An action is permissible if every precondition can unify with an expression in the current state. The function returns a list of dictionaries containing permissible substitutions or an empty list if there is not a valid unification for every value in the preconditions list.* **Used by**: [get_successors](#get_successors)

* **state** List[List[str]]: the current state
* **action** str: the name of the current action
* **action** Dict[str, Any]: the actions dictionary containing the action, preconditions, and add and delete rules

**returns** List[Dict[str, Any]]

In [6]:
def get_permissible_actions(state: List[List[str]], action: str, actions: Dict[str, Any]) -> List[Dict[str, Any]]:
    permissible_actions = []
    frontier = [{"substitutions": {}, "conditions": actions[action]["conditions"]}]
    while frontier:
        current_node = frontier.pop(0)
        if len(current_node["conditions"]) == 0:
            permissible_actions.append(current_node["substitutions"])
        for action in get_action_successors(current_node, state):
            frontier.append(action)
    return permissible_actions

In [7]:
#assertions/unit tests
expected_resp_1 = [
    {'?plane': '1973', '?to': 'SFO', '?from': 'SFO'}, 
    {'?plane': '1973', '?to': 'JFK', '?from': 'SFO'}, 
    {'?plane': '1973', '?to': 'ORD', '?from': 'SFO'}, 
    {'?plane': '2749', '?to': 'SFO', '?from': 'JFK'}, 
    {'?plane': '2749', '?to': 'JFK', '?from': 'JFK'}, 
    {'?plane': '2749', '?to': 'ORD', '?from': 'JFK'}, 
    {'?plane': '97', '?to': 'SFO', '?from': 'ORD'}, 
    {'?plane': '97', '?to': 'JFK', '?from': 'ORD'}, 
    {'?plane': '97', '?to': 'ORD', '?from': 'ORD'}, 
    {'?plane': '1211', '?to': 'SFO', '?from': 'SFO'}, 
    {'?plane': '1211', '?to': 'JFK', '?from': 'SFO'}, 
    {'?plane': '1211', '?to': 'ORD', '?from': 'SFO'}
]

assert get_permissible_actions(test_start_state, "fly", test_actions) == expected_resp_1
assert get_permissible_actions(test_start_state, "fuel", test_actions) == [{'?plane': '2749'}, {'?plane': '97'}]

# state with no fuel actions
test_state_2 = [
    ["plane", "1973"], 
    ["plane", "2749"], 
    ["plane", "97"], 
    ["plane", "1211"], 
    ["fueled", "1973"], 
    ["fueled", "2749"], 
    ["fueled", "97"], 
    ["fueled", "1211"]
]

assert get_permissible_actions(test_state_2, "fuel", test_actions) == []

<a id="get_next_state"></a>
## get_next_state
*The get_next_state function uses the add and delete lists for the current action and returns an updated state after applying the action.* **Used by**: [get_successors](#get_successors)

* **state** List[List[str]]: the current state
* **action** str: the name of the current action
* **substitutions** Dict[str, Any]: a dictionary containing the substitutions for a given action
* **action** Dict[str, Any]: the actions dictionary containing the action, preconditions, and add and delete rules

**returns** List[List[str]]

In [8]:
def get_next_state(state: List[List[str]], action: str, substitutions: Dict[str, Any], actions: Dict[str, Any]) -> List[List[str]]:
    new_state = deepcopy(state)
    
    # delete items on delete list
    for item_delete in actions[action].get("delete", []):
        remove = [substitutions[val] if is_variable(val) else val for val in item_delete]
        new_state = [value for value in new_state if value != remove]
        
    # add items on add list
    for item_add in actions[action].get("add", []):
        add = [substitutions[val] if is_variable(val) else val for val in item_add]
        if add not in new_state:
            new_state.append(add)
    return new_state

In [9]:
#assertions/unit tests
test_substitution_1 = {'?plane': '1973', '?to': 'JFK', '?from': 'SFO'}

expected_state_1 = [
    ['plane', '1973'], 
    ['plane', '2749'], 
    ['plane', '97'], 
    ['plane', '1211'], 
    ['airport', 'SFO'], 
    ['airport', 'JFK'], 
    ['airport', 'ORD'], 
    ['at', '2749', 'JFK'], 
    ['at', '97', 'ORD'], 
    ['at', '1211', 'SFO'], 
    ['fueled', '1973'], 
    ['unfueled', '2749'], 
    ['unfueled', '97'], 
    ['fueled', '1211'], 
    ['at', '1973', 'JFK']
]

assert get_next_state(test_start_state, "fly", test_substitution_1, test_actions) == expected_state_1

test_substitution_2 = {'?plane': '97'}

expected_state_2 = [
    ['plane', '1973'], 
    ['plane', '2749'], 
    ['plane', '97'], 
    ['plane', '1211'], 
    ['airport', 'SFO'], 
    ['airport', 'JFK'], 
    ['airport', 'ORD'], 
    ['at', '1973', 'SFO'], 
    ['at', '2749', 'JFK'], 
    ['at', '97', 'ORD'], 
    ['at', '1211', 'SFO'], 
    ['fueled', '1973'], 
    ['unfueled', '2749'], 
    ['fueled', '1211'],
    ['fueled', '97']
]

assert get_next_state(test_start_state, "fuel", test_substitution_2, test_actions) == expected_state_2

test_state_2 = [
    ['plane', '1973'], 
    ['plane', '2749'], 
    ['plane', '97'], 
    ['plane', '1211'],  
    ['fueled', '1973'], 
    ['fueled', '2749'], 
    ['fueled', '1211'],
    ['fueled', '97']
]

# no updates if th value is already in the explored list
assert get_next_state(test_state_2, "fuel", test_substitution_2, test_actions) == test_state_2

<a id="get_successors"></a>
## get_successors
*The get_successors function returns the successors of a current node. The main breadth-first search in the forward planner uses this function to get successor nodes. The function retrieves all permissible actions and uses these actions to update the state and path for the successor nodes. The function returns a list of dictionaries containing metadata about each successor.* **Used by**: [forward_planner](#forward_planner)

* **node** Dict[str, Any]: the current node of the form {"state": [], "previous": [], "path": []} where the state is the current state, previous is the previous state, and path is the current plan so far.
* **action** Dict[str, Any]: the actions dictionary containing the action, preconditions, and add and delete rules

**returns** List[dict[str, Any]]

In [10]:
def get_successors(node: Dict[str, Any], actions: Dict[str, Any]) -> List[Dict[str, Any]]:
    successors = []
    for action in actions:
        for substitution in get_permissible_actions(node["state"], action, actions):
            state = get_next_state(node["state"], action, substitution, actions)
            updated_action = ([substitution[val] if is_variable(val) else val for val in actions[action]["action"]])
            updated_path = node["path"] + [updated_action]
            successors.append({"state": state, "previous": node["previous"] + [node["state"]], "path": updated_path})
    return successors

In [11]:
#assertions/unit tests
test_state_1 = [
    ['plane', '1973'],  
    ['plane', '1211'],  
    ['fueled', '1973'], 
    ['unfueled', '1211'],
]

current_node_1 = {"state": test_state_1, "previous": [], "path": []}

expected_successors_1 = [
    {'state': [['plane', '1973'], ['plane', '1211'], ['fueled', '1973'], ['fueled', '1211']], 
     'previous': [[['plane', '1973'], ['plane', '1211'], ['fueled', '1973'], ['unfueled', '1211']]], 
     'path': [['fuel', '1211']]}
]

assert get_successors(current_node_1, test_actions) == expected_successors_1

# test with no permissible actions
test_state_2 = [
    ['plane', '1973'],  
    ['plane', '1211'],  
    ['fueled', '1973'], 
    ['fueled', '1211'],

]

current_node_2 = {"state": test_state_2, "previous": [], "path": []}

assert get_successors(current_node_2, test_actions) == []

test_state_3 = [
    ['plane', '1973'],  
    ['airport', 'SFO'], 
    ['airport', 'JFK'], 
    ['at', '1973', 'SFO'],  
    ['fueled', '1973']
]

current_node_3 = {"state": test_state_3, "previous": [], "path": []}

expected_successors_3 = [
    {'state': [['plane', '1973'], ['airport', 'SFO'], ['airport', 'JFK'], ['fueled', '1973'], ['at', '1973', 'SFO']], 
     'previous': [[['plane', '1973'], ['airport', 'SFO'], ['airport', 'JFK'], ['at', '1973', 'SFO'], ['fueled', '1973']]], 
     'path': [['fly', '1973', 'SFO', 'SFO']]}, 
    {'state': [['plane', '1973'], ['airport', 'SFO'], ['airport', 'JFK'], ['fueled', '1973'], ['at', '1973', 'JFK']], 
     'previous': [[['plane', '1973'], ['airport', 'SFO'], ['airport', 'JFK'], ['at', '1973', 'SFO'], ['fueled', '1973']]], 
     'path': [['fly', '1973', 'SFO', 'JFK']]}
]

assert get_successors(current_node_3, test_actions) == expected_successors_3

<a id="get_debug_plan"></a>
## get_debug_plan
*The get_debug_plan function returns a list of the form [s0, a1, s1, a2, s2, ...], where s is the state and a is an action in the plan. The forward planner prints the debug plan if the debug parameter is true.* **Used by**: [forward_planner](#forward_planner)

* **current_node** Dict[str, Any]: the current node contains a node of the form {"state": [], "previous": [], "path": []} where the state is the current state, previous is the previous state, and path is the current plan so far.

**returns** List[List[str]]

In [12]:
def get_debug_plan(current_node: Dict[str, Any]) -> List[List[str]]:
    debug_plan = [item for pair in zip(current_node["previous"], current_node["path"]) for item in pair]
    debug_plan.append(current_node["state"])
    return debug_plan

In [13]:
#assertions/unit tests
test_state_1 = [
    ['plane', '1973'],  
    ['airport', 'SFO'], 
    ['airport', 'JFK'], 
    ['at', '1973', 'SFO'],  
    ['fueled', '1973']
]

next_state_1 = [
    ['plane', '1973'],  
    ['airport', 'SFO'], 
    ['airport', 'JFK'], 
    ['at', '1973', 'JFK'],  
    ['unfueled', '1973']
]

test_node = {"state": next_state_1, "previous": [test_state_1, next_state_1], "path": [['fly', '1973', 'SFO', 'JFK']]}

expected_plan = [
    [['plane', '1973'], ['airport', 'SFO'], ['airport', 'JFK'], ['at', '1973', 'SFO'], ['fueled', '1973']], 
    ['fly', '1973', 'SFO', 'JFK'], 
    [['plane', '1973'], ['airport', 'SFO'], ['airport', 'JFK'], ['at', '1973', 'JFK'], ['unfueled', '1973']]
]

debug_plan = get_debug_plan(test_node)
assert debug_plan[0] == expected_plan[0]
assert debug_plan[1] == expected_plan[1]
assert debug_plan[2] == expected_plan[2]

<a id="forward_planner"></a>
## forward_planner
*The forward_planner function implements a forward planner using breadth-first search. The algorithm takes a starting state, a goal state, and a dictionary of actions. Each action contains a list of preconditions and add/delete lists that reflect how the current state will change when applying the action. The function returns a list of instantiated actions that will result in every goal condition existing in the current state. The function has an optional debug parameter that will print the action and the resulting state for each action in the plan.*

* **start_state** List[str]: the starting state containing a list of s-expression
* **goals** List[str]: the goal state containing a list of s-expressions
* **actions** Dict[str, Any]: the actions dictionary containing actions, preconditions, and add and delete rules for each action.
* **debug** bool: a boolean specifying if the function should print resulting states with the plan's actions
  
**returns** List[str]

In [14]:
def forward_planner(start_state: List[str], goal: List[str], actions: Dict[str, Any], debug:bool=False) -> List[str]:
    start_state, goals, actions = parse_values(start_state, goal, actions)
    frontier = [{"state": start_state, "previous": [], "path": []}]
    explored_list = [start_state]
    
    while len(frontier) > 0:
        current_state = frontier.pop(0)
        # return path if all goals are in the current state
        if all(val in current_state["state"] for val in goals):
            return get_debug_plan(current_state) if debug == True else current_state["path"]

        for successor in get_successors(current_state, actions):
            successor_state = successor["state"]
            if successor_state not in explored_list:
                frontier.append(successor)
                explored_list.append(successor_state)
    return None

In [15]:
#assertions/unit tests
test_start_state = [
    "(plane 1973)", 
    "(plane 2749)", 
    "(plane 97)",
    "(plane 1211)",
    "(airport SFO)", 
    "(airport JFK)", 
    "(airport ORD)",
    "(at 1973 SFO)",
    "(at 2749 JFK)",
    "(at 97 ORD)",
    "(at 1211 SFO)",
    "(fueled 1973)",
    "(unfueled 2749)",
    "(unfueled 97)",
    "(fueled 1211)" 
]

test_goals = [
    "(at 1973 ORD)",
    "(at 1211 JFK)", 
    "(at 2749 SFO)"
]

test_actions = {
    "fly": {
        "action": "(fly ?plane ?from ?to)",
        "conditions": [
            "(plane ?plane)", 
            "(airport ?to)",
            "(airport ?from)",
            "(at ?plane ?from)",
            "(fueled ?plane)"
        ],
        "add": [
            "(at ?plane ?to)",
            "(unfueled ?plane)"
        ],
        "delete": [
            "(at ?plane ?from)",
            "(fueled ?plane)"
        ]
    },
     "fuel": {
        "action": "(fuel ?plane)",
        "conditions": [
            "(plane ?plane)",
            "(unfueled ?plane)"
        ],
        "add": [
            "(fueled ?plane)"
        ],
        "delete": [
            "(unfueled ?plane)"
        ]
    }
}

expected_plan = [['fly', '1973', 'SFO', 'ORD'], ['fly', '1211', 'SFO', 'JFK'], ['fuel', '2749'], ['fly', '2749', 'JFK', 'SFO']]

assert forward_planner(test_start_state, test_goals, test_actions) == expected_plan

You will be solving the problem from above. Here is the start state:

In [16]:
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)"
]

The goal state:

In [17]:
goal = [
    "(item Saw)",    
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",    
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)"    
]

and the actions/operators:

In [18]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

## Print Plan

In [19]:
plan = forward_planner( start_state, goal, actions)

In [20]:
for el in plan:
    print(el)

['drive', 'Me', 'Home', 'Store']
['buy', 'Me', 'Store', 'Drill']
['drive', 'Me', 'Store', 'Home']


## Print Debug Plan

In [21]:
from pprint import pprint
plan = forward_planner( start_state, goal, actions, True)
for element in plan:
    pprint(element)

[['item', 'Saw'],
 ['item', 'Drill'],
 ['place', 'Home'],
 ['place', 'Store'],
 ['place', 'Bank'],
 ['agent', 'Me'],
 ['at', 'Me', 'Home'],
 ['at', 'Saw', 'Store'],
 ['at', 'Drill', 'Store']]
['drive', 'Me', 'Home', 'Store']
[['item', 'Saw'],
 ['item', 'Drill'],
 ['place', 'Home'],
 ['place', 'Store'],
 ['place', 'Bank'],
 ['agent', 'Me'],
 ['at', 'Saw', 'Store'],
 ['at', 'Drill', 'Store'],
 ['at', 'Me', 'Store']]
['buy', 'Me', 'Store', 'Drill']
[['item', 'Saw'],
 ['item', 'Drill'],
 ['place', 'Home'],
 ['place', 'Store'],
 ['place', 'Bank'],
 ['agent', 'Me'],
 ['at', 'Saw', 'Store'],
 ['at', 'Me', 'Store'],
 ['at', 'Drill', 'Me']]
['drive', 'Me', 'Store', 'Home']
[['item', 'Saw'],
 ['item', 'Drill'],
 ['place', 'Home'],
 ['place', 'Store'],
 ['place', 'Bank'],
 ['agent', 'Me'],
 ['at', 'Saw', 'Store'],
 ['at', 'Drill', 'Me'],
 ['at', 'Me', 'Home']]
